In [1]:
import torchaudio
import matplotlib.pyplot as plt
import numpy as np
from math import floor
from IPython.display import display,Audio
import torch
from pathlib import Path

In [2]:
class CFG:
    device='cuda'
    RATE=16000
    percent=0.9
    batch_size=4
    epoches=20
    lr=5e-7
    load_pth=None
    debug=114514
    save_steps=5
    length=15360

In [3]:
snaps=[]
un_snaps=[]

In [4]:
def Load_Snaps(folder = "./audios/snaps/",fuck=False):
    if(fuck):
        filenames=[folder]
    else:
        PATH = Path(folder)
        filenames=list(PATH.glob('*.wav'))
    sum=0
    for filename in filenames:
        waveform,sample_rate = torchaudio.load(filename)
        # print("Shape of waveform:{}".format(waveform.size())) #音频大小
        # print("sample rate of waveform:{}".format(sample_rate))#采样率
        resampler = torchaudio.transforms.Resample(sample_rate, CFG.RATE)
        if(sample_rate!=CFG.RATE):
            waveform=resampler(waveform)
        for channel in waveform:
            if(fuck):
                return channel
            if(len(channel)!=CFG.length):
                print(len(channel))
                continue
            snaps.append(channel)
            sum+=1
    print("Load {0} snaps".format(sum))

In [5]:
def Load_un_Snaps(filename = "./audios/un_snaps.wav"):
    cut_steps=CFG.length
    waveform,sample_rate = torchaudio.load(filename)
    tot_size=waveform.size()
    print("Shape of waveform:{}".format(tot_size)) #音频大小
    print("sample rate of waveform:{}".format(sample_rate))#采样率
    resampler = torchaudio.transforms.Resample(sample_rate, CFG.RATE)
    if(sample_rate!=CFG.RATE):
        waveform=resampler(waveform)
    sum=0
    for channel in waveform:
        data_=channel
        for i in range(0,int(floor(data_.shape[0]/cut_steps))):
            data=data_[i*cut_steps:(i+1)*cut_steps]
            if(len(data)!=CFG.length):
                continue
            un_snaps.append(data)
            sum+=1
    print("Load {0} un_snaps".format(sum))

In [6]:
Load_Snaps()
Load_un_Snaps("audios/un_snaps_1.wav")
Load_un_Snaps("audios/un_snaps_2.wav")
# Load_un_Snaps("audios/un_snaps_3.wav")
Load_un_Snaps("audios/un_snaps_4.wav")

Load 480 snaps
Shape of waveform:torch.Size([2, 11614208])
sample rate of waveform:48000
Load 504 un_snaps
Shape of waveform:torch.Size([2, 29532160])
sample rate of waveform:48000
Load 1280 un_snaps
Shape of waveform:torch.Size([1, 1920000])
sample rate of waveform:16000
Load 125 un_snaps


In [7]:
from torch.utils.data import DataLoader,Dataset
class SnapSet(Dataset):
    def __init__(self,snaps,un_snaps):
        self.inputs=[]
        self.labels=[]
        for (i,snap) in enumerate(snaps):
            self.inputs.append(snap)
            self.labels.append(1)

        for (i,un_snap) in enumerate(un_snaps):
            self.inputs.append(un_snap)
            self.labels.append(0)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, item):
        # print(item)
        return self.inputs[item].to(CFG.device),\
        torch.as_tensor(self.labels[item],device=CFG.device,dtype=torch.float32)

In [8]:
import random
random.shuffle(snaps)
random.shuffle(un_snaps)
tot=int(len(snaps)*CFG.percent)
# tot_=int(len(un_snaps)*CFG.percent)
left=len(snaps)-tot
train_set=SnapSet(snaps[:tot],un_snaps[:2*tot])
val_set=SnapSet(snaps[-left:],un_snaps[-left*2:])
train_loader=DataLoader(train_set,batch_size=CFG.batch_size,shuffle=True,drop_last=True)
val_loader=DataLoader(val_set,batch_size=CFG.batch_size,shuffle=True,drop_last=True)
print(len(train_loader))

324


In [9]:
from transformers import Wav2Vec2FeatureExtractor, WavLMForXVector
pretrained = WavLMForXVector.from_pretrained("microsoft/wavlm-base-plus")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("microsoft/wavlm-base-plus")
pretrained=pretrained.to(CFG.device)

Some weights of WavLMForXVector were not initialized from the model checkpoint at microsoft/wavlm-base-plus and are newly initialized: ['tdnn.0.kernel.bias', 'classifier.bias', 'tdnn.1.kernel.weight', 'feature_extractor.bias', 'classifier.weight', 'tdnn.2.kernel.weight', 'projector.bias', 'tdnn.4.kernel.weight', 'tdnn.3.kernel.bias', 'tdnn.0.kernel.weight', 'objective.weight', 'feature_extractor.weight', 'projector.weight', 'tdnn.3.kernel.weight', 'tdnn.1.kernel.bias', 'tdnn.4.kernel.bias', 'tdnn.2.kernel.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from torch import nn
class Snap(nn.Module):
    def __init__(self):
        super(Snap,self).__init__()
        sample_snap=Load_Snaps('audios/snaps/snap0.wav',fuck=True)
        self.sample_inputs=feature_extractor(sample_snap, padding=True, return_tensors="pt",sampling_rate=CFG.RATE).to(CFG.device)
        # print(self.sample_inputs['input_values'].shape)

    def forward(self, x):
        x=x.reshape(CFG.batch_size,-1).cpu().numpy()
        tmp=[]
        for a in x:
            tmp.append(a)
        x=feature_extractor(tmp, padding=True, return_tensors="pt",sampling_rate=CFG.RATE).to(CFG.device)
        self.sample=pretrained(**self.sample_inputs).embeddings[0]
        x=pretrained(**x).embeddings
        cosine_sim = torch.nn.CosineSimilarity(dim=-1)
        similarity = cosine_sim(self.sample, x)
        return similarity

if CFG.load_pth!=None:
    pretrained=WavLMForXVector.from_pretrained(CFG.load_pth)

model=Snap()
model=model.to(CFG.device)

In [11]:
def Mean(a):
    s=0
    t=0
    for _ in a:
        s+=_
        t+=1
    return s/t

def Accuracy(out,labels):
    a=0
    b=0
    for j in range(len(labels)):
        b+=1
        if (out[j].item()-0.86)*(labels[j].item()-0.5)>0:
            a+=1
    return a/b

In [12]:
from transformers import AdamW
from tqdm import tqdm

optimizer=AdamW(pretrained.parameters(),lr=CFG.lr)

accuracy_max=-114514
loss_tot=None

for epoch in range(CFG.epoches):
    model.train()
    for i,batch in tqdm(enumerate(train_loader)):
        inputs,labels=batch
        # print(len(inputs))
        # print(labels)
        out=model(inputs)
        labels=labels.view(-1)
        loss=((out-labels)**2).mean()
        loss.backward()
        optimizer.step()
        model.zero_grad()
        if (i+1)%CFG.debug==0:
            accuracy=Accuracy(out,labels)
            print("#Training epoch:{0} idx={1} loss={2} accuracy={3}".format(epoch, i, loss.item(),accuracy))
    
    loss_=[]
    accuracy_=[]
    model.eval()
    model.zero_grad()
    for i,batch in enumerate(val_loader):
        with torch.no_grad():
            inputs,labels=batch
            out=model(inputs)
            labels=labels.view(-1)
            loss=((out-labels)**2).mean()
            accuracy=Accuracy(out,labels)
            loss_.append(loss)
            accuracy_.append(accuracy)

    loss_=Mean(loss_)
    accuracy_=Mean(accuracy_)
    print("#Validation epoch:{0} loss={1} accuracy={2}".format(epoch,loss_,accuracy_))

    if(accuracy_>accuracy_max):
        accuracy_max=accuracy_
        print("Saving best model")
        pretrained.save_pretrained("./pth/best_pth")

    if((epoch+1)%CFG.save_steps==0):
        print("Saving {0} model".format(epoch+1))
        pretrained.save_pretrained("./pth/{0}_pth".format(epoch))

c:\Users\Hasee\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
0it [00:00, ?it/s]c:\Users\Hasee\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\models\wavlm\modeling_wavlm.py:1755: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
324it [01:06,  4.88it/s]


#Validation epoch:0 loss=0.020237378776073456 accuracy=0.9791666666666666
Saving best model


324it [01:04,  5.01it/s]


#Validation epoch:1 loss=0.008714120835065842 accuracy=1.0
Saving best model


324it [01:08,  4.72it/s]


#Validation epoch:2 loss=0.055924467742443085 accuracy=0.9513888888888888


282it [01:04,  4.37it/s]